In [2]:
# dependencies
"""
Numpy: matrix manipulation and math
Pandas: csv parsing and various data structure tasks
Mathpltlib.pyplot: data visualization
set_trace: debug breaks
keras: a machine learning library that is intuitive to read
"""
from copy import deepcopy as copy
from IPython.core.debugger import set_trace

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import os
import tensorflow as tf
#from tensorflow import keras 
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

tf.config.optimizer.set_jit(True)

from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [3]:
"""
concat_files
----------

Concatenate text files in a directory as a string

dependent on 'os' Python module

parameters
----------
directory: string; path of the target directory

f_type:    tuple of strings; target file extentsions Ex: ('.py', '.cpp')

return
---------
content:   concatenated string

"""
def concat_files(directory,f_type):
    import os
    # List all file in the dataset directory
    # ------------------
    all_file = []
    content = ""

    # walk through every directory and open every f_type file
    # concatenate into var string "content"
    for root, dirs, files in os.walk(directory): 
        for name in files:
            if name.endswith(f_type): # we only care about .py
                all_file.append(name)
                with open(os.path.join(root,name), "r") as f:
                    content += f.read() + "\n"
    return content

In [4]:
content = concat_files("dataset",('.py'))

In [5]:
"""
encode_string
-----------
Generate a dictionary representation of the characters found 
in a string keyed with integer representations

Returns two dictionaries and an array. The two dictionaries are 
necessary to convert the string to integer representation
and back again. The array is the string encoded as integer values.

parameters
----------
content:      string; to be processed

return
----------
vocab_to_int: dict; character to integer representation of unique characters in the string

int_to_vocab: dict; integer to string representation

encoded:      array; string encoded as integer values
"""

def encode_string(content):   
    # Convert the string "content" into a list of intergers
    
    # creates a set of the individual characters
    vocab = set(content) 
    
    # use the set to sequentially generate a dictionary
    vocab_to_int = {c: i for i, c in enumerate(vocab)} 
    
    # make keys the numerical values
    int_to_vocab = dict(enumerate(vocab)) 
    
    # encode the "content" string using dict
    encoded = np.array([vocab_to_int[c] for c in content], dtype=np.int32)
    
    return vocab_to_int, int_to_vocab, encoded

In [6]:
vocab_to_int, int_to_vocab, encoded = encode_string(content)

In [7]:
print(content)
print(int_to_vocab)
# this is all of the files concatenated. with each character encoded using the int_to_vocab
print(encoded)

"""
PyTest's for Digital Image Processing
"""

import digital_image_processing.edge_detection.canny as canny
import digital_image_processing.filters.gaussian_filter as gg
import digital_image_processing.filters.median_filter as med
import digital_image_processing.filters.sobel_filter as sob
import digital_image_processing.filters.convolve as conv
import digital_image_processing.change_contrast as cc
import digital_image_processing.convert_to_negative as cn
from cv2 import imread, cvtColor, COLOR_BGR2GRAY
from numpy import array, uint8
from PIL import Image

img = imread(r"digital_image_processing/image_data/lena_small.jpg")
gray = cvtColor(img, COLOR_BGR2GRAY)

# Test: convert_to_negative()
def test_convert_to_negative():
    negative_img = cn.convert_to_negative(img)
    # assert negative_img array for at least one True
    assert negative_img.any()


# Test: change_contrast()
def test_change_contrast():
    with Image.open("digital_image_processing/image_data/lena_small.jpg") as img:

In [8]:
segment = []
for i in range(16): 
    segment.append((int(len(encoded) / 16) * (i + 1)))
batch = {
    "x" : [
        encoded[:segment[0]],
        encoded[segment[0]:segment[1]]
    ],
    "y" : [
        encoded[segment[1]:segment[2]],
        encoded[segment[2]:segment[3]]
    ]
}

In [9]:
# inp = np.vstack((batch['x'][0],batch['x'][1]))
# targets = np.vstack((batch['y'][0],batch['y'][1]))
inp = batch['x'][0]
targets = batch['y'][0]
io_size = len(int_to_vocab)

inp = tf.one_hot(inp,io_size)
targets = tf.one_hot(targets,io_size)

print(targets)

tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(67995, 133), dtype=float32)


In [109]:
# lines = []
# temp = []
# for i, char in enumerate(encoded):
#     if int_to_vocab[char] == '\n':
#         lines.append(temp)
#     else:
#         temp.append(char)
#         temp = []

# x = np.zeros((len(lines), maxlen, len(int_to_vocab)), dtype=np.bool)
# y = np.zeros((len(sentences), len(int_to_vocab)), dtype=np.bool)
# for i, sentence in enumerate(sentences):
#     for t, char in enumerate(sentence):
#         x[i, t, char_indices[char]] = 1
#     y[i, char_indices[next_chars[i]]] = 1

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.optimizers import RMSprop

# x_train = np.hstack((batch['x'][0],batch['x'][1]))
# y_train = np.hstack((batch['y'][0],batch['y'][1]))
# x_test = np.hstack((batch['x'][0],batch['x'][1]))
# y_test = np.hstack((batch['y'][0],batch['y'][1]))

max_features = 1024
#maxlen = 40

model = Sequential()
model.add(Embedding(max_features, output_dim=256))
model.add(LSTM(128, input_shape=inp.shape))
model.add(Dropout(0.5))
model.add(Dense(io_size, activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.fit(inp, targets, steps_per_epoch=1, epochs=10)
score = model.evaluate(inp, targets, steps_per_epoch=1)

/home/bran/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10


In [24]:
with tf.name_scope("LSTM"):

    def create_cell():
        lstm = tf.contrib.rnn.BasicLSTMCell(4)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop

    cell = tf.contrib.rnn.MultiRNNCell([create_cell() for _ in range(3)])
    initial_state = cell.zero_state(2, tf.float32)

    io_size = len(vocab)
    x_one_hot = tf.one_hot(inputs, io_size)
    cell_outputs, final_state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state)

AttributeError: module 'tensorflow' has no attribute 'contrib'